## Bulk Transfer
Transfer classified TDE and AGN from GROWTH Marshal to Fritz

In [2]:
import os
import json
import requests
import pickle
import numpy as np
import pandas as pd
from copy import deepcopy
from astropy.table import Table
from astropy.time import Time
import astropy.io.ascii as asci
from astropy import units as u
from astropy.coordinates import SkyCoord

In [3]:
import matplotlib
import matplotlib.pyplot as plt
fs = 10
matplotlib.rcParams['font.size']=fs
%matplotlib notebook

### Step 1. Get classifications from Growth Marshal

In [4]:
f=open("json_dumps/ZTFBH_Nuclear_2020-11-25.json")
data = json.load(f)
f.close()

In [6]:
print (len(data))

10296


In [167]:
print (data[123]["name"])
print (data[123]["classification"])

ZTF18abrzbnb
AGN


### Step 2. Save Source to Fritz

In [153]:
global GETTOKEN, BASEURL

In [142]:
f = open("files/fritz_token_classify") # the ACLS of this token needs to be 'Classify'
GETTOKEN = f.readline().split("\n")[0]
f.close()

In [182]:
BASEURL = 'https://fritz.science/'

def api(method, endpoint, data=None):
    ''' 
    Info : Basic API query, takes input the method (eg. GET, POST, etc.), the endpoint (i.e. API url) 
               and additional data for filtering
    Returns : response in json format
    '''
    headers = {'Authorization': f'token {GETTOKEN}'}
    response = requests.request(method, endpoint, json=data, headers=headers)
    #if(response.json()['status'] == 'success'):
    #    pass
    #else:
    #    print('ERROR, Query failed!')
    return response.json()


def get_source_api(ztfname):
    ''' 
    Info : Query a single source, takes input ZTF name
    Returns : all basic data of that source (excludes photometry and spectra, 
                includes redshift, classification, comments, etc.)
    '''
    url = BASEURL+'api/sources/'+ztfname
    response = api('GET',url)
    return response['data']    

def get_group_ids(groupnames=['Nuclear Transients']):
    ''' 
    Info : Query group ids of groups
    Returns : List of group ids 
    '''
    url = BASEURL+'api/groups'
    headers = {'Authorization': f'token {GETTOKEN}'}
    groupnames = np.atleast_1d(groupnames)
    grpids = []
    for grpname in groupnames:
        response = requests.request('GET',url,params={'name':grpname}, headers=headers).json()
        grpids.append(response['data'][0]['id'])
    return grpids


def save_alert_to_group(ztfname, groupname):
    ''' 
    Info : Save an alert to given group(s), takes the ZTF name and list of group names
    Returns : response with status
    '''
    url = BASEURL+'api/alerts/ztf/'+ztfname
    filt = {'group_ids':get_group_ids(groupname)}
    headers = {'Authorization': f'token {GETTOKEN}'}
    response = requests.request('POST', url, json=filt, headers=headers)
    return response.json()


def post_classification(ztfname, classification, probability=1.0):
    '''
    Info : Post a classification, takes input ZTF name, classification and taxonomy_id 
           (1 for sitewide, 2 for basic). Only classifications within that taxonomy are permitted.
            Check Fritz API query to list all taxonomies in documentation
    '''
    url = BASEURL+'api/classification'
    filt = {"obj_id":ztfname,
            'classification':classification,
            'taxonomy_id':1,
            "probability":probability
            #'group_ids':get_group_ids() #default to all of requesting user's groups.
           }
    print(filt)
    response = api('POST',url,filt)
    return response

### taxonomy

In [127]:
url = BASEURL+'api/taxonomy'
response = api('GET',url)

In [129]:
# response["data"]

### save source 

In [147]:
get_group_ids(groupnames=['Nuclear Transients'])

[80]

In [171]:
info1 = get_source_api("ZTF18absopva")
info2 = get_source_api("ZTF18abrzbnb")

In [174]:
info1=={} # this means that a source had already been saved before

False

In [176]:
info2=={} # this means that s source had never been saved, in this case, we need to save it

True

In [177]:
q = save_alert_to_group("ZTF18abrzbnb",'Nuclear Transients')

In [178]:
q

{'status': 'success',
 'data': {'id': 'ZTF18abrzbnb'},
 'version': '0.9.dev0+git20201125.95d9e76+fritz.7d7a638'}

### post classfication

In [179]:
info1 = get_source_api("ZTF18absopva")
info2 = get_source_api("ZTF18abrzbnb")

In [185]:
info1["classifications"]==[]

False

In [186]:
info2["classifications"]==[]

True

In [187]:
post_classification("ZTF18abrzbnb", "AGN")

{'obj_id': 'ZTF18abrzbnb', 'classification': 'AGN', 'taxonomy_id': 1, 'probability': 1.0}


{'status': 'success',
 'data': {'classification_id': 9066},
 'version': '0.9.dev0+git20201125.95d9e76+fritz.7d7a638'}